In [ ]:
!pip install opencv-python

In [ ]:
#join the owners duet session
import syft as sy
sy.VERBOSE = True
duet = sy.join_duet(loopback=True)

In [ ]:
#check the items in the duet store
duet.store.pandas

In [ ]:
#store the item in duet1_ptr
data1_ptr = duet.store[0]
print(data1_ptr)

In [ ]:
import torch
import time

start = time.time()

In [ ]:
#set the input and output dimensions
in_dim = 1
out_dim = 1

In [ ]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.linear = self.torch_ref.nn.Linear(in_dim, out_dim)

    def forward(self, x):
        x = self.linear(x)
        return x

In [ ]:
combined_model = SyNet(torch)

In [ ]:
def train(iterations, model, torch_ref, optim, data_ptr, target_ptr):
    
    losses = []
    
    for i in range(iterations):
        
        optim.zero_grad()
        print("check 1")
        
        output = model(data_ptr)
        print("check 2")

        loss = torch_ref.nn.functional.mse_loss(output, target_ptr)
        print("check 3")

        loss_item = loss.item()
        print(type(loss_item))
        print("check 4")
        loss_value = loss_item.get(
                name="loss",
                reason="To evaluate training progress",
                request_block=True,
                timeout_secs=5
            )
        print("check 5")
        if i % 10 == 0:
            print("Epoch", i, "loss", loss_value)
        losses.append(loss_value)
        loss.backward()
        optim.step()
        print("check 6")
    
    print ("Process took: ", time.time()-start, " seconds")
    return losses

In [ ]:
import torch as th
import numpy as np

In [ ]:
local_model1 = SyNet(torch)


In [ ]:
print(local_model1.parameters())


In [ ]:
remote_model1 = local_model1.send(duet)


In [ ]:
remote_torch1 = duet.torch
params = remote_model1.parameters()
optim1 = remote_torch1.optim.Adam(params=params, lr=0.1)

In [ ]:
from skimage.transform import resize
from skimage.color import rgb2gray
#from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import torch
import os
import imageio
import cv2 as cv
#loop through the directory and transform to tensor
path = "/Users/danielcastillo/Documents/senior-project/Data/CATS_DOGS/train/CAT"

arr = []
from tqdm.notebook import tqdm
tqdm().pandas()
i = 0


for pic in tqdm(os.scandir(path)):
    if i < 40:
        if (pic.path.endswith(".jpeg") or pic.path.endswith(".jpg")) and pic.is_file():
            picture = imageio.imread(pic.path)
            #check if rgba
            if(picture is None):
                print(i, "not added to list")
                continue
            else:
                #resize the picture
                picture = resize(picture, (1,1), mode='constant', anti_aliasing=True)
                #change the picture to gray scale
                picture = rgb2gray(picture)
                #transform the picture to a tensor
                Tensor = torch.tensor(picture)
                Tensor = Tensor.flatten()
                #append the tensor to the list
                arr.append(Tensor)
                print(i, "added to list")
    i+=1

In [ ]:
for index in arr:
    print(type(index))

In [ ]:
target1_ptr = torch.stack(arr)
target1_ptr = target1_ptr.type(torch.FloatTensor)
print(target1_ptr)
print(target1_ptr.shape)

In [ ]:
for index in target1_ptr:
    print(type(index))

In [ ]:
print(target1_ptr.dtype)
print(type(target1_ptr))

In [ ]:
iteration = 100
losses = train(iteration, remote_model1, remote_torch1, optim1, data1_ptr, target1_ptr)